<a href="https://colab.research.google.com/github/germancruzram/EXCEL-TO-KML/blob/main/XLS_a_KML_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CUADERNO EN COLAB PARA CREAR UN KML PARTIENDO DE UNA TABLA EXCEL CON COORDENADAS GEOGRAFICAS EN GRADOS DECIMALES.**

**✍ German Cruz Ramírez. 🇳**

# **1. PREPAREMOS LAS LIBRERIAS A UTILIZAR**

In [10]:
!pip install simplekml
!pip install folium
!pip install gdal
import pandas as pd
import simplekml
import os
import json

# **2. CREAREMOS LAS CARPETAS XLS Y KML**

In [11]:
carpeta_xls = "XLS"
carpeta_kml = "KML"


if not os.path.exists(carpeta_xls):
    os.makedirs(carpeta_xls)
    print(f'Creada la carpeta "{carpeta_xls}, ahora puedes subir la tabla con coordenadas"')
else:
    print(f'La carpeta "{carpeta_xls}" ya existe y puedes subir la tabla con coordenadas')

if not os.path.exists(carpeta_kml):
    os.makedirs(carpeta_kml)
    print(f'Creada la carpeta "{carpeta_kml}"')
else:
    print(f'La carpeta "{carpeta_kml}" ya existe')


La carpeta "XLS" ya existe y puedes subir la tabla con coordenadas
La carpeta "KML" ya existe


**💡 Recuerda que la tabla excel debe contener el campo "Latitud" y "Longitud**

# **3. PROCESANDO LA INFORMACION PARA CREAR EL KML**

In [17]:
import os
import pandas as pd
import simplekml

carpeta_xls = "/content/XLS"
carpeta_kml = "/content/KML"
nombre_campo = input("Por favor, ingresa el nombre del campo que quiere visualizar: ")
archivos_xls = [archivo for archivo in os.listdir(carpeta_xls) if archivo.endswith(('.xlsx', '.xls', '.csv'))]

for nombre_archivo in archivos_xls:
    ruta_archivo = os.path.join(carpeta_xls, nombre_archivo)

    # Verificar si el archivo es CSV o Excel y cargarlo en un DataFrame
    if nombre_archivo.endswith('.csv'):
        df = pd.read_csv(ruta_archivo)
    else:
        df = pd.read_excel(ruta_archivo)

    # Crear un nuevo archivo KML
    kml = simplekml.Kml()

    # Obtener valores únicos del campo especificado para crear carpetas
    valores_unicos = df[nombre_campo].dropna().unique()

    # Crear carpetas en KML para cada valor único en el campo especificado
    for valor in valores_unicos:
        carpeta = kml.newfolder(name=str(valor))
        df_valor = df[df[nombre_campo] == valor]

        # Agregar puntos en la carpeta específica
        for idx, fila in df_valor.iterrows():
            descripcion = '<br>'.join([f'{col}: {fila[col]}' for col in df.columns if col not in ['Latitud', 'Longitud']])
            nombre_punto = fila.get(nombre_campo, "Sin información")
            punto = carpeta.newpoint(name=nombre_punto, coords=[(fila['Longitud'], fila['Latitud'])])
            punto.description = descripcion

    # Guardar el archivo KML
    nombre_kml = nombre_archivo.rsplit('.', 1)[0] + '.kml'
    ruta_kml = os.path.join(carpeta_kml, nombre_kml)
    kml.save(ruta_kml)
    print(f"Archivo KML guardado en: {ruta_kml}")



Por favor, ingresa el nombre del campo que quiere visualizar: Deterioro


ValueError: Excel file format cannot be determined, you must specify an engine manually.

# **4. CREAR UNA VISTA GIS EN COLAB**

In [13]:
import folium
import os
from osgeo import ogr

def kml_to_geojson(kml_path, geojson_path):
    driver = ogr.GetDriverByName('KML')
    data = driver.Open(kml_path)
    if data is None:
        raise Exception(f"Error al abrir el archivo KML: {kml_path}")
    layer = data.GetLayer()
    driver = ogr.GetDriverByName('GeoJSON')
    if os.path.exists(geojson_path):
        driver.DeleteDataSource(geojson_path)
    outData = driver.CreateDataSource(geojson_path)
    outLayer = outData.CreateLayer('layer', geom_type=ogr.wkbMultiPolygon)
    layerDefn = layer.GetLayerDefn()
    for i in range(layerDefn.GetFieldCount()):
        fieldDefn = layerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)
    outLayerDefn = outLayer.GetLayerDefn()
    for feature in layer:
        geom = feature.GetGeometryRef()
        outFeature = ogr.Feature(outLayerDefn)
        outFeature.SetGeometry(geom)
        for i in range(0, outLayerDefn.GetFieldCount()):
            outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), feature.GetField(i))
        outLayer.CreateFeature(outFeature)
        outFeature = None
    outData = None

carpeta_kml = "/content/KML"

archivos_kml = [archivo for archivo in os.listdir(carpeta_kml) if archivo.endswith('.kml')]
if not archivos_kml:
    raise Exception("No se encontraron archivos KML en la carpeta especificada.")

nombre_kml = archivos_kml[0]
nombre_geojson = nombre_kml.replace('.kml', '.geojson')
ruta_kml = os.path.join(carpeta_kml, nombre_kml)
ruta_geojson = os.path.join(carpeta_kml, nombre_geojson)
kml_to_geojson(ruta_kml, ruta_geojson)
m = folium.Map(location=[0, 0], zoom_start=2)

def add_popups(feature):
    properties = feature['properties']
    html = '<br>'.join([f'{key}: {value}' for key, value in properties.items()])
    return folium.Popup(html)

folium.GeoJson(
    ruta_geojson,
    name="GeoJSON data",
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['Name', 'Description'],
        aliases=['Nombre:', 'Descripción:']
    ),
    popup=add_popups
).add_to(m)

folium.LayerControl().add_to(m)
m

# **5. BORRAR EL CONTENIDO PARA PROCESAR OTRA TABLA EXCEL**

In [ ]:
from pathlib import Path

def eliminar_contenido_carpeta(path):
    carpeta = Path(path)

    if carpeta.exists():
        [elemento.unlink() for elemento in carpeta.iterdir() if elemento.is_file() or elemento.is_symlink()]

carpeta_kml = "/content/KML"
carpeta_xls = "/content/XLS"

eliminar_contenido_carpeta(carpeta_kml)
eliminar_contenido_carpeta(carpeta_xls)

print("El contenido de las carpetas KML y XLS ha sido eliminado.")


El contenido de las carpetas KML y XLS ha sido eliminado.
